In [3]:
from GoogleTools.ReadSheet import load_gsheet
import pandas as pd

In [6]:
tmp = load_gsheet('1_M93Kbidv6_ekdV6Ji3EXG2TwIbc65pFFkvg0zPLYhY', 'account!B1:L508')
df = pd.DataFrame(data=tmp[1:], columns=tmp[0])
dct = {k:v for k,v in zip(df.Name, df.focus)}
print(df.shape)
df.head(2)

TypeError: load_gsheet() missing 1 required positional argument: 'range_name'

In [2]:
from fuzzywuzzy.process import extractOne
from tqdm import tqdm
import numpy as np

count_file = '../data/dashboard-downloads/pages-by-most-hs-count.csv'
percent_file = '../data/dashboard-downloads/pages-by-most-hs-percentage.csv'


def find_focus(filename, N=100, agg=sum):
    hscounttop = pd.read_csv(filename, nrows=N)
    hscounttop.columns = ['page_group', 'weight']
    matches = pd.DataFrame({
        'dash_name': hscounttop.page_group,
        'weight': hscounttop.weight,
        'match': [extractOne(q, df.Name) for q in tqdm(hscounttop.page_group)]
    })
    matches['final_match'] = [m[0] if m[1]>=90 else '' for m in matches.match]
    matches['focus'] = [dct[m] if len(m)>0 else '' for m in matches.final_match]
    print('\nNumber of Pages/Groups Not Found:', sum([len(m)==0 for m in matches.focus]))
    matches = matches.loc[[len(f)>0 and f!='myanmar' for f in matches.focus]]
    return matches, matches.groupby('focus').final_match.count().sort_values(ascending=False), matches.groupby('focus').weight.agg(agg).sort_values(ascending=False)


ModuleNotFoundError: No module named 'fuzzywuzzy'

In [ ]:
c, c1, c2 = find_focus(count_file, agg=sum)

In [ ]:
p, p1, p2 = find_focus(percent_file, agg=np.mean)

In [ ]:
c1

In [ ]:
c2

In [ ]:
p1

In [ ]:
p2

In [ ]:
dx = pd.DataFrame({
    'hscount_count': c1,
    'hscount_weight': c2,
    'hspercent_count': p1,
    'hspercent_weight': p2
})

In [ ]:
dx.to_csv('../data/dashboard-downloads/top-focus.csv')